Import agate for some agate time

In [28]:
import agate

Pull in the 2013 lunch information csv, clean out any empty values, print table columns and lengths

In [29]:
lunches2013 = agate.Table.from_csv('frl13.csv')
cleanLunches2013 = lunches2013.where(lambda row: row['FREEREDUCED13'] is not None)
print(cleanLunches2013)
print(len(cleanLunches2013.rows))

|----------------+---------------|
|  column_names  | column_types  |
|----------------+---------------|
|  FY13          | Date          |
|  COUNTY13      | Text          |
|  CODISTSCH13   | Number        |
|  SCHOOLNAME13  | Text          |
|  SCHOOLTYPE13  | Text          |
|  GRADE13       | Text          |
|  ENROLLED13    | Number        |
|  FREEREDUCED13 | Number        |
|  PERCENT13     | Number        |
|  WARNING13     | Text          |
|----------------+---------------|

1075


Pull in the 2014 lunch information csv, clean out any empty values, print table columns and lengths

In [3]:
lunches2014 = agate.Table.from_csv('frl14.csv')
cleanLunches2014 = lunches2014.where(lambda row: row['FREEREDUCED14'] is not None)
print(cleanLunches2014)
print(len(cleanLunches2014.rows))

|----------------+---------------|
|  column_names  | column_types  |
|----------------+---------------|
|  FY14          | Date          |
|  COUNTY14      | Text          |
|  CODISTSCH14   | Number        |
|  SCHOOLNAME14  | Text          |
|  SCHOOLTYPE14  | Text          |
|  GRADE14       | Text          |
|  ENROLLED14    | Number        |
|  FREEREDUCED14 | Number        |
|  PERCENT14     | Number        |
|  WARNING14     | Text          |
|  CEP14         | Text          |
|----------------+---------------|

1052


Pull in the 2015 lunch information csv, clean out any empty values, print table columns and lengths

In [30]:
lunches2015 = agate.Table.from_csv('frl15.csv')
cleanLunches2015 = lunches2015.where(lambda row: row['FREEREDUCED15'] is not None)
print(cleanLunches2015)
print(len(cleanLunches2015.rows))

|------------------+---------------|
|  column_names    | column_types  |
|------------------+---------------|
|  FY15            | Date          |
|  COUNTY15        | Text          |
|  CODISTSCH15     | Number        |
|  SCHOOLNAME15    | Text          |
|  SCHOOLTYPE15    | Text          |
|  GRADE15         | Text          |
|  ENROLLED15      | Number        |
|  FREEREDUCED15   | Number        |
|  PERCENT15       | Number        |
|  WARNING15       | Text          |
|  CEP_Indicator15 | Text          |
|------------------+---------------|

1047


Join the cleanLunches2013 & 2014 on district codes

In [8]:
lunches1314 = cleanLunches2013.join(cleanLunches2014, 'CODISTSCH13', 'CODISTSCH14', inner=True )

View the new joined table

In [9]:
print(lunches1314)
print(len(lunches1314.rows))

|----------------+---------------|
|  column_names  | column_types  |
|----------------+---------------|
|  FY13          | Date          |
|  COUNTY13      | Text          |
|  CODISTSCH13   | Number        |
|  SCHOOLNAME13  | Text          |
|  SCHOOLTYPE13  | Text          |
|  GRADE13       | Text          |
|  ENROLLED13    | Number        |
|  FREEREDUCED13 | Number        |
|  PERCENT13     | Number        |
|  WARNING13     | Text          |
|  FY14          | Date          |
|  COUNTY14      | Text          |
|  SCHOOLNAME14  | Text          |
|  SCHOOLTYPE14  | Text          |
|  GRADE14       | Text          |
|  ENROLLED14    | Number        |
|  FREEREDUCED14 | Number        |
|  PERCENT14     | Number        |
|  WARNING14     | Text          |
|  CEP14         | Text          |
|----------------+---------------|

1026


Final join between joined lunches1314 and lunches2015

In [12]:
lunches = lunches1314.join(cleanLunches2015, 'CODISTSCH13', 'CODISTSCH15', inner=True )

View the final joined table

In [13]:
print(lunches)
print(len(lunches.rows))

|------------------+---------------|
|  column_names    | column_types  |
|------------------+---------------|
|  FY13            | Date          |
|  COUNTY13        | Text          |
|  CODISTSCH13     | Number        |
|  SCHOOLNAME13    | Text          |
|  SCHOOLTYPE13    | Text          |
|  GRADE13         | Text          |
|  ENROLLED13      | Number        |
|  FREEREDUCED13   | Number        |
|  PERCENT13       | Number        |
|  WARNING13       | Text          |
|  FY14            | Date          |
|  COUNTY14        | Text          |
|  SCHOOLNAME14    | Text          |
|  SCHOOLTYPE14    | Text          |
|  GRADE14         | Text          |
|  ENROLLED14      | Number        |
|  FREEREDUCED14   | Number        |
|  PERCENT14       | Number        |
|  WARNING14       | Text          |
|  CEP14           | Text          |
|  FY15            | Date          |
|  COUNTY15        | Text          |
|  SCHOOLNAME15    | Text          |
|  SCHOOLTYPE15    | Text          |
|

Compute the percent changes for both the free lunch enrollment and enrollment change for comparsion

In [14]:
change = lunches.compute([
    ('freelunch_change', agate.PercentChange('FREEREDUCED13', 'FREEREDUCED15')),
    ('enrollment_change', agate.PercentChange('ENROLLED13', 'ENROLLED15'))        
])

Make those percent changes to one decimal

In [18]:
from decimal import Decimal
def round_freelunch_change(row):
    return row['freelunch_change'].quantize(Decimal('0.1'))
def round_enrollment_change(row):
    return row['enrollment_change'].quantize(Decimal('0.1'))
    
rounded_change = change.compute([
    ('pc_freelunch', agate.Formula(agate.Number(), round_freelunch_change)),
    ('pc_enrollment', agate.Formula(agate.Number(), round_enrollment_change)),
])

Find those top free, reduced lunch percent change

In [19]:
sorted_change = rounded_change.order_by('pc_freelunch', reverse=True)

Only need certain columns to print out

In [26]:
for_printing = sorted_change.select(['SCHOOLNAME13', 'pc_freelunch', 'pc_enrollment'])

Print out the final table

In [27]:
for_printing.print_table()

|-------------------------------------------------+--------------+----------------|
|  SCHOOLNAME13                                   | pc_freelunch | pc_enrollment  |
|-------------------------------------------------+--------------+----------------|
|  EARLY CHILDHOOD LEARNING CENTER                |        400.0 |          27.6  |
|  ALLEN ELEMENTARY SCHOOL                        |        178.6 |         -10.8  |
|  ROHWER ELEMENTARY SCHOOL                       |        157.1 |          31.4  |
|  CEDAR BLUFFS MIDDLE SCHOOL                     |        143.8 |          61.0  |
|  ST PAUL LUTHERAN ELEM SCHOOL                   |        133.3 |          17.0  |
|  CEDAR BLUFFS PRESCHOOL                         |        118.2 |         122.2  |
|  KENESAW SECONDARY SCHOOL                       |        113.3 |          17.3  |
|  GARDEN COUNTY ELEMENTARY SCH                   |        108.0 |          52.5  |
|  LOURDES CENTRAL MIDDLE SCHOOL                  |        100.0 |          